In [1]:
from census import Census
from us import states
import uszipcode
import pandas as pd

from data_compilation import census_key
import json

import requests
import matplotlib.pyplot as plt
import numpy as numpy



ModuleNotFoundError: No module named 'data_compilation'

In [3]:
file_path_census = "Resources/wa_census.csv"
clean_census = pd.read_csv(file_path_census)
print(clean_census)



      Zipcode ID  Census Year  WA Zipcode  Median Household Income  \
0              0         2020       98901                    45062   
1              1         2020       98237                    58063   
2              2         2020       98201                    59357   
3              3         2020       98021                   129215   
4              4         2020       98203                    80556   
...          ...          ...         ...                      ...   
1541        1541         2022       99348                    74211   
1542        1542         2022       98609               -666666666   
1543        1543         2022       98336                    76250   
1544        1544         2022       98935                    60085   
1545        1545         2022       99179                    78906   

      Income Per Capita  Population  
0               22108.0       31425  
1               29153.0        4762  
2               34624.0       30645  
3      

In [4]:
# Assuming 'Census Year' is the column representing the year in the Census DataFrame
census_df_2022 = clean_census[clean_census['Census Year'] == 2022]

# Display the filtered DataFrame
print(census_df_2022)


      Zipcode ID  Census Year  WA Zipcode  Median Household Income  \
1029        1029         2022       98901                    55536   
1030        1030         2022       98237                    68929   
1031        1031         2022       98201                    71313   
1032        1032         2022       98021                   155891   
1033        1033         2022       98203                    96351   
...          ...          ...         ...                      ...   
1541        1541         2022       99348                    74211   
1542        1542         2022       98609               -666666666   
1543        1543         2022       98336                    76250   
1544        1544         2022       98935                    60085   
1545        1545         2022       99179                    78906   

      Income Per Capita  Population  
1029            25237.0       33106  
1030            31650.0        4979  
1031            41730.0       30587  
1032   

In [5]:
census_df_2022_byzips = census_df_2022.groupby('WA Zipcode').value_counts()


print(census_df_2022_byzips)

##This is Census Dataframe we want to use and merge with the EV population of each zip code

WA Zipcode  Zipcode ID  Census Year  Median Household Income  Income Per Capita  Population
98001       1048        2022         103338                   44341.0            35842         1
98002       1097        2022         68040                    33306.0            36812         1
98003       1178        2022         67547                    34070.0            51329         1
98004       1115        2022         157784                   117764.0           39161         1
98005       1151        2022         151297                   87533.0            22030         1
                                                                                              ..
99360       1263        2022         72115                    26768.0            1400          1
99361       1506        2022         81250                    37923.0            1668          1
99362       1142        2022         63778                    35063.0            43028         1
99402       1440        2022       

In [6]:
#want to find how many EVs are in each zip code, then if that correlates to the population of that zipcode

In [7]:
file_path = "Resources/Clean_EV_Data.csv"
clean_EV = pd.read_csv(file_path)
clean_EV


,Unnamed: 0,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,2,JN1AZ0CP8B,Yakima,Yakima,WA,98901,2011,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,15.0,218972519,POINT (-120.50721 46.60448),PACIFICORP,53077001602
1,3,1G1FW6S08H,Skagit,Concrete,WA,98237,2017,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,39.0,186750406,POINT (-121.7515 48.53892),PUGET SOUND ENERGY INC,53057951101
2,4,3FA6P0SU1K,Snohomish,Everett,WA,98201,2019,FORD,FUSION,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26,0,38.0,2006714,POINT (-122.20596 47.97659),PUGET SOUND ENERGY INC,53061041500
3,5,5YJ3E1EB5J,Snohomish,Bothell,WA,98021,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,1.0,475635324,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,53061051916
4,6,1N4AZ0CP4D,Snohomish,Everett,WA,98203,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,38.0,253546023,POINT (-122.23019 47.94949),PUGET SOUND ENERGY INC,53061040900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112343,112629,7SAYGDEF2N,King,Duvall,WA,98019,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,45.0,217955265,POINT (-121.98609 47.74068),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033032401
112344,112630,1N4BZ1CP7K,San Juan,Friday Harbor,WA,98250,2019,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,150,0,40.0,103663227,POINT (-123.01648 48.53448),BONNEVILLE POWER ADMINISTRATION||ORCAS POWER &...,53055960301
112345,112631,1FMCU0KZ4N,King,Vashon,WA,98070,2022,FORD,ESCAPE,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,34.0,193878387,POINT (-122.4573 47.44929),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033027702
112346,112632,KNDCD3LD4J,King,Covington,WA,98042,2018,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26,0,47.0,125039043,POINT (-122.09124 47.33778),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033032007


In [21]:
clean_EV_byzips = clean_EV.groupby('Postal Code')

clean_EV_byzips

In [10]:
clean_zip = clean_EV['Postal Code']
zip_list = list(clean_zip.unique())
# zip_list

print(zip_list)

[98901, 98237, 98201, 98021, 98203, 98275, 98236, 98221, 98516, 98837, 98579, 98233, 98344, 98271, 98037, 98026, 99026, 98502, 98087, 98125, 98001, 98115, 98020, 98260, 98503, 98501, 98208, 98296, 98312, 98036, 98204, 98902, 98908, 99163, 99205, 98392, 98272, 98382, 98345, 98513, 98038, 98270, 98031, 98155, 98370, 98290, 98052, 98029, 98632, 98406, 98926, 98168, 98329, 98373, 98074, 98102, 98595, 98683, 98011, 99001, 98075, 98040, 98292, 98053, 98110, 98032, 98112, 98122, 98116, 98002, 98822, 98498, 98391, 98106, 98402, 98008, 98663, 99212, 98033, 98144, 98446, 98107, 98059, 98367, 98225, 98661, 99354, 98004, 98282, 98801, 98103, 98178, 98258, 98684, 98686, 98229, 98580, 98332, 98101, 98198, 98177, 98607, 99338, 98136, 98366, 98105, 98058, 98012, 98166, 99352, 98118, 98604, 98685, 98121, 99362, 98045, 98006, 98034, 98056, 98506, 98030, 98274, 98335, 98005, 98674, 98072, 98524, 98119, 98065, 98024, 98374, 98250, 98117, 98281, 98327, 98109, 98077, 98611, 98230, 98682, 99336, 98027, 99208

In [11]:

# Use value_counts() to get counts of each unique postal code
zip_counts = clean_zip.value_counts()

# Print the counts
print(zip_counts)
#see the zip codes with the most EVs


Postal Code
98052    2916
98033    2059
98004    2001
98115    1880
98006    1852
         ... 
98263       1
98326       1
98283       1
98530       1
99179       1
Name: count, Length: 528, dtype: int64


In [12]:
# Find the mode (most frequent) zip code
mode_zip = zip_counts.idxmax()

# Print the mode zip code
print("Mode Postal Code:", mode_zip)

#look to see where zipcode 98052 is located
#Maybe we can look into the statistical data, to show a geographic preference?


Mode Postal Code: 98052


In [72]:
#Create new dataframe with only selected columns
#Our two dataframes are "census_df_2022_byzips" and "clean_EV_byzips"



zipcode_list = []
income_census = []
income_pc_census = []
census_year = []
wa_population = []


# Replace 'your_api_key_here' with your actual Census API key
# api_key = '2dd7e9076df3078c080c3f8749e8231f1501927d'
# Create a Census connection
years = ['2022']
for year in years:
    print(year)
    base_url = f'https://api.census.gov/data/{year}/acs/acs5?get='
    variables = 'NAME,B19013_001E,B19301_001E,B01003_001E'
    for zipcode in zipcode_list:
        print(zipcode)
        url = f"{base_url}{variables}&for=zip%20code%20tabulation%20area:{zipcode}&key={census_key}"
        
        try:
            response = requests.get(url).json()
            zipcode_list.append(response[1][4])
            income_census.append(response[1][1])
            income_pc_census.append(response[1][2])
            census_year.append(year)
            wa_population.append(response[1][3])
            print(response)
        except: 
            print('zipcode not found')
            pass
#example
# https://api.census.gov/data/2022/acs/acs5?get=NAME,B01001_001E&for=zip%20code%20tabulation%20area:08701&key=YOUR_KEY_GOES_HERE


2022
98901
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98901', '55536', '25237', '33106', '98901']]
98237
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98237', '68929', '31650', '4979', '98237']]
98201
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98201', '71313', '41730', '30587', '98201']]
98021
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98021', '155891', '67017', '34993', '98021']]
98203
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98203', '96351', '44288', '36448', '98203']]
98275
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98275', '123036', '62514', '21365', '98275']]
98236
[['NAME', 'B19013_001E', 'B19301_001E', 'B01003_001E', 'zip code tabulation area'], ['ZCTA5 98236', '91328', '47720', '6790', '982

In [73]:
census_df = pd.DataFrame({
    'Census Year': census_year,
    'Postal Code': zipcode_list,
    'Median Household Income': income_census,
    'Income Per Capita': income_pc_census,
    'Population': wa_population
})
census_df

ValueError: All arrays must be of the same length

In [ ]:
census_df.to_csv('Resources/wa_census.csv', index_label='WA Zipcode')

In [13]:
census_file = 'Resources/wa_census.csv'
census_df = pd.read_csv(census_file)
census_2022 = census_df.loc[census_df['Census Year'] == 2022]
census_2022 = census_2022.loc[census_2022['Median Household Income'] > 0]
census_2022.dropna()
census_2022

,Zipcode ID,Census Year,WA Zipcode,Median Household Income,Income Per Capita,Population
1029,1029,2022,98901,55536,25237.0,33106
1030,1030,2022,98237,68929,31650.0,4979
1031,1031,2022,98201,71313,41730.0,30587
1032,1032,2022,98021,155891,67017.0,34993
1033,1033,2022,98203,96351,44288.0,36448
...,...,...,...,...,...,...
1540,1540,2022,99131,34081,31798.0,122
1541,1541,2022,99348,74211,32056.0,1590
1543,1543,2022,98336,76250,30549.0,594
1544,1544,2022,98935,60085,18814.0,4378


In [25]:
# Rename the column
census_2022 = census_2022.rename(columns={'WA Zipcode': 'Postal Code'})
census_2022.head()

# Save to CSV with the new column name
census_2022.to_csv('Resources/wa_census.csv', index_label='Postal Code')

In [29]:
merged_df = census_2022.merge(clean_EV, how = 'outer', on = 'Postal Code')

merged_df.to_csv('Resources/merged.csv', index_label = 'Postal Code')


In [35]:
sorted_df = merged_df.sort_values(by='Postal Code')
sorted_df

,Zipcode ID,Census Year,Postal Code,Median Household Income,Income Per Capita,Population,Unnamed: 0,VIN (1-10),County,City,...,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
7570,1048.0,2022.0,98001,103338.0,44341.0,35842.0,53275,3FA6P0SU6F,King,Auburn,...,FUSION,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,19,0,30.0,135831601,POINT (-122.23035 47.3074),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033029901
7510,1048.0,2022.0,98001,103338.0,44341.0,35842.0,36854,7SAYGDEF1N,King,Auburn,...,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,30.0,207623629,POINT (-122.23035 47.3074),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033030405
7511,1048.0,2022.0,98001,103338.0,44341.0,35842.0,36873,LPSED3KA5N,King,Auburn,...,PS2,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,47.0,190146785,POINT (-122.23035 47.3074),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033029901
7512,1048.0,2022.0,98001,103338.0,44341.0,35842.0,37232,7SAYGAEEXN,King,Algona,...,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,30.0,185717794,POINT (-122.23035 47.3074),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033030901
7513,1048.0,2022.0,98001,103338.0,44341.0,35842.0,37372,KNDCD3LD9N,King,Auburn,...,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26,0,47.0,203146562,POINT (-122.23035 47.3074),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033029902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107166,1312.0,2022.0,99403,60657.0,35383.0,20425.0,63256,5YJSA1E64M,Asotin,Clarkston,...,MODEL S,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,9.0,161650394,POINT (-117.04556 46.41402),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,53003960500
107142,1312.0,2022.0,99403,60657.0,35383.0,20425.0,1777,WBY1Z8C57H,Asotin,Clarkston,...,I3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,97,0,9.0,6029299,POINT (-117.04556 46.41402),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,53003960400
107164,1312.0,2022.0,99403,60657.0,35383.0,20425.0,56542,7SAYGDEE9N,Asotin,Clarkston,...,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,9.0,224361854,POINT (-117.04556 46.41402),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,53003960500
107162,1312.0,2022.0,99403,60657.0,35383.0,20425.0,48149,1N4AZ0CP8D,Asotin,Clarkston,...,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,9.0,234328636,POINT (-117.04556 46.41402),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,53003960500
